# SETUP

In [1]:
%whos

Interactive namespace is empty.


In [2]:
import os
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import re
import matplotlib.pyplot as plt
% matplotlib inline

from matplotlib import rcParams

rcParams["font.family"] = 'Nanum Gothic'
rcParams["font.size"] = 20
rcParams["figure.figsize"] = (14,4)

from konlpy.tag import Komoran
komoran = Komoran()

ㅑ

In [3]:
import gc
gc.collect()

8

In [60]:
import matplotlib.font_manager as fm

In [61]:
print(rcParams['font.size'])
print(rcParams['font.family'])

20.0
['Nanum Gothic']


In [62]:
cs_raw = pd.read_pickle('cs_whole.pkl')
cs_raw.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


In [63]:
corpus = ""
texts = cs_raw.body
for text in texts:
    corpus += text
corpus = re.sub('\n', ' ', corpus)

In [ ]:
# corpus_refined = normalize(corpus, english=True, number=True)
# len(corpus), len(corpus_refined)

In [18]:
import sys
sys.path.insert(0, '../')
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length=10,
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
All 2398 r features was loaded


In [27]:
sentence_origin = corpus.split('.')

In [28]:
len(sentence_origin)

23287

In [31]:
nouns_origin = noun_extractor.train_extract(
    sentence_origin,
    min_noun_score=0.3,
    min_noun_frequency=20
)

scanning completed
(L,R) has (5289, 2779) tokens
building lr-graph completed

In [42]:
type(corpus), type(nouns_origin)

(str, dict)

In [48]:
texts = ""
for text in list(nouns_origin)[:5]:
    texts += text
    texts += " "
texts[:10]

'자질 정면 최소 주'

### 워드 클라우드

In [34]:
stopwords = []

In [49]:
# Create and Generate a word cloud image:
wc = WordCloud(max_font_size=50, max_words=100, font_path='/Library/Fonts/AppleMyungjo.ttf', background_color="white").generate(texts)

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()
fig.savefig('wordcloud_without_axisoff.png')

### text to sparse

In [57]:
from collections import defaultdict
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
vectorizer = CountVectorizer(
    min_df=0.005,
    max_df=0.95
)

x_sparse = vectorizer.fit_transform(texts)

### keyword_extraction

In [74]:
x_sparse

<1022x7557 sparse matrix of type '<class 'numpy.int64'>'
	with 163973 stored elements in Compressed Sparse Row format>

In [83]:
documents = lassobased_extractor.get_document_index('박근혜')

ValueError: You should set index2word first

In [78]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(min_tf=20, min_df=10)
lassobased_extractor.train(x_sparse)

In [82]:
keywords = lassobased_extractor.extract_from_docs(texts, min_num_of_keywords=10)

There is no corresponding documents
